# HW3 SPARK

In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType, DateType, ArrayType, FloatType

In [2]:
spark = SparkSession.builder \
.master("local[2]") \
.appName("CleanDataStepByStep") \
.getOrCreate()

2023-02-22 23:40:35,106 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-02-22 23:40:36,015 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Download Data

In [3]:
! wget -P /home/train/datasets/ \
https://github.com/erkansirin78/datasets/raw/master/dirty_store_transactions.csv

--2023-02-22 23:44:03--  https://github.com/erkansirin78/datasets/raw/master/dirty_store_transactions.csv
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/erkansirin78/datasets/master/dirty_store_transactions.csv [following]
--2023-02-22 23:44:04--  https://raw.githubusercontent.com/erkansirin78/datasets/master/dirty_store_transactions.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2609524 (2.5M) [text/plain]
Saving to: ‘/home/train/datasets/dirty_store_transactions.csv’

100%[======================================>] 2,609,524   5.90MB/s   in 0.4s   

2023-02-22 23:44:06 (5.90 MB/s) - ‘/home

## Data Cleaning Preps

In [4]:
df = spark.read \
.format("csv") \
.option("header", True) \
.option("sep", ",") \
.option("inferSchema", True) \
.load("file:///home/train/datasets/dirty_store_transactions.csv")

In [5]:
df.show()

+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|    CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|  YR7220|     New York(|     Electronics|  12254943|$31|$20.77|   $1.86|$29.14|2019-11-26|
|  YR7220|     New York+|       Furniture| 72619323C|$15| $9.75|    $1.5| $13.5|2019-11-26|
|  YR7220|     New York |     Electronics| 34161682B|$88|$62.48|    $4.4| $83.6|2019-11-26|
|  YR7220|     New York!|         Kitchen|  79411621|$91|$58.24|   $3.64|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion| 39520263T|$85|   $51|   $2.55|$82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204|$37|$24.05|   $0.74|$36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics| 86610412D|$80| $48.8|    $6.4| $73.6|2019-11-26|
|  YR7220|     New York$|         Kitchen| 52503356^|$71| $42.6|   $5.68|$65.32|

In [6]:
df.printSchema()

root
 |-- STORE_ID: string (nullable = true)
 |-- STORE_LOCATION: string (nullable = true)
 |-- PRODUCT_CATEGORY: string (nullable = true)
 |-- PRODUCT_ID: string (nullable = true)
 |-- MRP: string (nullable = true)
 |-- CP: string (nullable = true)
 |-- DISCOUNT: string (nullable = true)
 |-- SP: string (nullable = true)
 |-- Date: string (nullable = true)



In [7]:
df1=df.withColumn("Date", F.to_date("Date", "y-M-d"))

In [8]:
df1.show()

+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|    CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|  YR7220|     New York(|     Electronics|  12254943|$31|$20.77|   $1.86|$29.14|2019-11-26|
|  YR7220|     New York+|       Furniture| 72619323C|$15| $9.75|    $1.5| $13.5|2019-11-26|
|  YR7220|     New York |     Electronics| 34161682B|$88|$62.48|    $4.4| $83.6|2019-11-26|
|  YR7220|     New York!|         Kitchen|  79411621|$91|$58.24|   $3.64|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion| 39520263T|$85|   $51|   $2.55|$82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204|$37|$24.05|   $0.74|$36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics| 86610412D|$80| $48.8|    $6.4| $73.6|2019-11-26|
|  YR7220|     New York$|         Kitchen| 52503356^|$71| $42.6|   $5.68|$65.32|

In [9]:
df1.printSchema()

root
 |-- STORE_ID: string (nullable = true)
 |-- STORE_LOCATION: string (nullable = true)
 |-- PRODUCT_CATEGORY: string (nullable = true)
 |-- PRODUCT_ID: string (nullable = true)
 |-- MRP: string (nullable = true)
 |-- CP: string (nullable = true)
 |-- DISCOUNT: string (nullable = true)
 |-- SP: string (nullable = true)
 |-- Date: date (nullable = true)



In [10]:
df2=df1.withColumn("CP", F.translate("CP", "$", " ")) \
.withColumn("DISCOUNT",  F.translate("DISCOUNT", "$", " ")) \
.withColumn("SP",  F.translate("SP", "$", " ")) \
.withColumn("MRP",  F.translate("MRP", "$", " "))

In [11]:
df2.show()

+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|    CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|  YR7220|     New York(|     Electronics|  12254943| 31| 20.77|    1.86| 29.14|2019-11-26|
|  YR7220|     New York+|       Furniture| 72619323C| 15|  9.75|     1.5|  13.5|2019-11-26|
|  YR7220|     New York |     Electronics| 34161682B| 88| 62.48|     4.4|  83.6|2019-11-26|
|  YR7220|     New York!|         Kitchen|  79411621| 91| 58.24|    3.64| 87.36|2019-11-26|
|  YR7220|      New York|         Fashion| 39520263T| 85|    51|    2.55| 82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204| 37| 24.05|    0.74| 36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics| 86610412D| 80|  48.8|     6.4|  73.6|2019-11-26|
|  YR7220|     New York$|         Kitchen| 52503356^| 71|  42.6|    5.68| 65.32|

In [12]:
df3=df2.withColumn("CP", F.col("CP").cast(FloatType())) \
.withColumn("DISCOUNT",  F.col("DISCOUNT").cast(FloatType())) \
.withColumn("SP",  F.col("SP").cast(FloatType())) \
.withColumn("MRP",  F.col("MRP").cast(IntegerType())) 

In [13]:
df3.printSchema()

root
 |-- STORE_ID: string (nullable = true)
 |-- STORE_LOCATION: string (nullable = true)
 |-- PRODUCT_CATEGORY: string (nullable = true)
 |-- PRODUCT_ID: string (nullable = true)
 |-- MRP: integer (nullable = true)
 |-- CP: float (nullable = true)
 |-- DISCOUNT: float (nullable = true)
 |-- SP: float (nullable = true)
 |-- Date: date (nullable = true)



In [14]:
df3.show()

+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|   CP|DISCOUNT|   SP|      Date|
+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|  YR7220|     New York(|     Electronics|  12254943| 31|20.77|    1.86|29.14|2019-11-26|
|  YR7220|     New York+|       Furniture| 72619323C| 15| 9.75|     1.5| 13.5|2019-11-26|
|  YR7220|     New York |     Electronics| 34161682B| 88|62.48|     4.4| 83.6|2019-11-26|
|  YR7220|     New York!|         Kitchen|  79411621| 91|58.24|    3.64|87.36|2019-11-26|
|  YR7220|      New York|         Fashion| 39520263T| 85| 51.0|    2.55|82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204| 37|24.05|    0.74|36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics| 86610412D| 80| 48.8|     6.4| 73.6|2019-11-26|
|  YR7220|     New York$|         Kitchen| 52503356^| 71| 42.6|    5.68|65.32|2019-11-26|
|  YR7220|

In [15]:
df4=df3.withColumn("STORE_LOCATION", F.regexp_replace(F.col("STORE_LOCATION"), "[^a-zA-Z0-9]", "")) \
.withColumn("PRODUCT_ID", F.regexp_replace(F.col("PRODUCT_ID"), "[^a-zA-Z0-9]", ""))

regexp_replace("STORE_LOCATION", "[^0-9a-zA-Z_\-]+", "").alias('STORE_LOCATION')

In [16]:
df4.show()

+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|   CP|DISCOUNT|   SP|      Date|
+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|  YR7220|       NewYork|     Electronics|  12254943| 31|20.77|    1.86|29.14|2019-11-26|
|  YR7220|       NewYork|       Furniture| 72619323C| 15| 9.75|     1.5| 13.5|2019-11-26|
|  YR7220|       NewYork|     Electronics| 34161682B| 88|62.48|     4.4| 83.6|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  79411621| 91|58.24|    3.64|87.36|2019-11-26|
|  YR7220|       NewYork|         Fashion| 39520263T| 85| 51.0|    2.55|82.45|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  93809204| 37|24.05|    0.74|36.26|2019-11-26|
|  YR7220|       NewYork|       Cosmetics| 86610412D| 80| 48.8|     6.4| 73.6|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  52503356| 71| 42.6|    5.68|65.32|2019-11-26|
|  YR7220|

In [17]:
df5=df4.select([F.col(x).alias(x.lower()) for x in df1.columns])

In [18]:
df5.printSchema()

root
 |-- store_id: string (nullable = true)
 |-- store_location: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- mrp: integer (nullable = true)
 |-- cp: float (nullable = true)
 |-- discount: float (nullable = true)
 |-- sp: float (nullable = true)
 |-- date: date (nullable = true)



## Data Cleaning

In [19]:
import findspark
findspark.init("/opt/manual/spark/")

In [20]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType, DateType, ArrayType, FloatType

In [ ]:
spark = SparkSession.builder \
.master("local[2]") \
.enableHiveSupport() \
.appName("CleanData") \
.getOrCreate()

In [ ]:
! wget -P /home/train/datasets/ \
https://github.com/erkansirin78/datasets/raw/master/dirty_store_transactions.csv

In [ ]:
df = spark.read \
.format("csv") \
.option("header", True) \
.option("sep", ",") \
.option("inferSchema", True) \
.load("file:///home/train/datasets/dirty_store_transactions.csv")

In [21]:
df1=df.withColumn("Date", F.to_date("Date", "y-M-d")) \
.withColumn("CP", F.translate("CP", "$", "").cast(DoubleType())) \
.withColumn("DISCOUNT",  F.translate("DISCOUNT", "$", "").cast(FloatType())) \
.withColumn("SP",  F.translate("SP", "$", "").cast(FloatType())) \
.withColumn("MRP",  F.translate("MRP", "$", "").cast(IntegerType())) \
.withColumn("STORE_LOCATION", F.regexp_replace(F.col("STORE_LOCATION"), "[^a-zA-Z0-9]", "")) \
.withColumn("PRODUCT_ID", F.regexp_replace(F.col("PRODUCT_ID"), "[^0-9]", ""))

In [22]:
df1.printSchema()

root
 |-- STORE_ID: string (nullable = true)
 |-- STORE_LOCATION: string (nullable = true)
 |-- PRODUCT_CATEGORY: string (nullable = true)
 |-- PRODUCT_ID: string (nullable = true)
 |-- MRP: integer (nullable = true)
 |-- CP: double (nullable = true)
 |-- DISCOUNT: float (nullable = true)
 |-- SP: float (nullable = true)
 |-- Date: date (nullable = true)



In [23]:
df1.show()

+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|   CP|DISCOUNT|   SP|      Date|
+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|  YR7220|       NewYork|     Electronics|  12254943| 31|20.77|    1.86|29.14|2019-11-26|
|  YR7220|       NewYork|       Furniture|  72619323| 15| 9.75|     1.5| 13.5|2019-11-26|
|  YR7220|       NewYork|     Electronics|  34161682| 88|62.48|     4.4| 83.6|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  79411621| 91|58.24|    3.64|87.36|2019-11-26|
|  YR7220|       NewYork|         Fashion|  39520263| 85| 51.0|    2.55|82.45|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  93809204| 37|24.05|    0.74|36.26|2019-11-26|
|  YR7220|       NewYork|       Cosmetics|  86610412| 80| 48.8|     6.4| 73.6|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  52503356| 71| 42.6|    5.68|65.32|2019-11-26|
|  YR7220|

In [24]:
df2=df1.select([F.col(x).alias(x.lower()) for x in df1.columns])

In [25]:
df2.show()

+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|store_id|store_location|product_category|product_id|mrp|   cp|discount|   sp|      date|
+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|  YR7220|       NewYork|     Electronics|  12254943| 31|20.77|    1.86|29.14|2019-11-26|
|  YR7220|       NewYork|       Furniture|  72619323| 15| 9.75|     1.5| 13.5|2019-11-26|
|  YR7220|       NewYork|     Electronics|  34161682| 88|62.48|     4.4| 83.6|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  79411621| 91|58.24|    3.64|87.36|2019-11-26|
|  YR7220|       NewYork|         Fashion|  39520263| 85| 51.0|    2.55|82.45|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  93809204| 37|24.05|    0.74|36.26|2019-11-26|
|  YR7220|       NewYork|       Cosmetics|  86610412| 80| 48.8|     6.4| 73.6|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  52503356| 71| 42.6|    5.68|65.32|2019-11-26|
|  YR7220|

In [27]:
df2.createOrReplaceTempView("data_temp")

In [28]:
df_check = spark.sql("""
    
    
    SELECT STORE_LOCATION, SUM(CP), SUM(DISCOUNT), SUM(SP)
    FROM data_temp 
    GROUP BY STORE_LOCATION;

""")

In [29]:
df_check.show()

+--------------+------------------+------------------+------------------+
|STORE_LOCATION|           sum(CP)|     sum(DISCOUNT)|           sum(SP)|
+--------------+------------------+------------------+------------------+
|    Washington|324520.73999999854| 29375.87996855378|450798.11982536316|
|       Houston| 278819.5199999993| 24945.19997137785|388271.79981708527|
|       NewYork| 298466.4899999972| 26283.11000086367| 415423.8898162842|
|         Miami|267729.23999999836|24296.639976918697|371295.35949897766|
|        Denver|244569.53999999826|21501.479988992214| 340802.5197200775|
+--------------+------------------+------------------+------------------+



In [30]:
df.count()

37853

## Write Clean Data into Hive

In [31]:
hive_df=df1.select([F.col(x).alias(x.lower()) for x in df1.columns])

In [32]:
hive_df.printSchema()

root
 |-- store_id: string (nullable = true)
 |-- store_location: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- mrp: integer (nullable = true)
 |-- cp: double (nullable = true)
 |-- discount: float (nullable = true)
 |-- sp: float (nullable = true)
 |-- date: date (nullable = true)



In [33]:
hive_df.show()

+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|store_id|store_location|product_category|product_id|mrp|   cp|discount|   sp|      date|
+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|  YR7220|       NewYork|     Electronics|  12254943| 31|20.77|    1.86|29.14|2019-11-26|
|  YR7220|       NewYork|       Furniture|  72619323| 15| 9.75|     1.5| 13.5|2019-11-26|
|  YR7220|       NewYork|     Electronics|  34161682| 88|62.48|     4.4| 83.6|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  79411621| 91|58.24|    3.64|87.36|2019-11-26|
|  YR7220|       NewYork|         Fashion|  39520263| 85| 51.0|    2.55|82.45|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  93809204| 37|24.05|    0.74|36.26|2019-11-26|
|  YR7220|       NewYork|       Cosmetics|  86610412| 80| 48.8|     6.4| 73.6|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  52503356| 71| 42.6|    5.68|65.32|2019-11-26|
|  YR7220|

In [34]:
hive_df.count()

37853

In [35]:
spark.sql("CREATE DATABASE IF NOT EXISTS test1")

DataFrame[]

In [36]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
|    test1|
+---------+



### ORC Format

In [37]:
hive_df.write.format("orc") \
.mode("overwrite") \
.saveAsTable("test1.clean_transactions1")

In [38]:
!hdfs dfs -ls /user/hive/warehouse/test1.db/clean_transactions1

Found 2 items
-rw-r--r--   1 train hive          0 2023-02-23 00:11 /user/hive/warehouse/test1.db/clean_transactions1/_SUCCESS
-rw-r--r--   1 train hive     194855 2023-02-23 00:11 /user/hive/warehouse/test1.db/clean_transactions1/part-00000-3a888059-3bb5-495b-9e9c-a06b10ded976-c000.snappy.orc


### Default Format: Parquet

In [40]:
hive_df.write \
.mode("overwrite") \
.saveAsTable("test1.clean_transactions_default")

In [41]:
!hdfs dfs -ls /user/hive/warehouse/test1.db/clean_transactions_default

Found 2 items
-rw-r--r--   1 train hive          0 2023-02-23 00:13 /user/hive/warehouse/test1.db/clean_transactions_default/_SUCCESS
-rw-r--r--   1 train hive     241687 2023-02-23 00:13 /user/hive/warehouse/test1.db/clean_transactions_default/part-00000-0607339a-45a7-493d-84bd-a94258d409eb-c000.snappy.parquet


### TXT Format

In [42]:
hive_df.write \
.mode("overwrite") \
.option("fileFormat", "textfile") \
.saveAsTable("test1.clean_transactions_text")

In [43]:
!hdfs dfs -ls /user/hive/warehouse/test1.db/clean_transactions_text

Found 2 items
-rw-r--r--   1 train hive          0 2023-02-23 00:14 /user/hive/warehouse/test1.db/clean_transactions_text/_SUCCESS
-rw-r--r--   1 train hive     241687 2023-02-23 00:14 /user/hive/warehouse/test1.db/clean_transactions_text/part-00000-aba641d1-2ab4-4cce-967c-a191f1b82843-c000.snappy.parquet


## Read Data from Hive

In [44]:
df_read1 = spark.read.table("test1.clean_transactions1")

In [45]:
df_read1.show()

+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|store_id|store_location|product_category|product_id|mrp|   cp|discount|   sp|      date|
+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|  YR7220|       NewYork|     Electronics|  12254943| 31|20.77|    1.86|29.14|2019-11-26|
|  YR7220|       NewYork|       Furniture|  72619323| 15| 9.75|     1.5| 13.5|2019-11-26|
|  YR7220|       NewYork|     Electronics|  34161682| 88|62.48|     4.4| 83.6|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  79411621| 91|58.24|    3.64|87.36|2019-11-26|
|  YR7220|       NewYork|         Fashion|  39520263| 85| 51.0|    2.55|82.45|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  93809204| 37|24.05|    0.74|36.26|2019-11-26|
|  YR7220|       NewYork|       Cosmetics|  86610412| 80| 48.8|     6.4| 73.6|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  52503356| 71| 42.6|    5.68|65.32|2019-11-26|
|  YR7220|

In [46]:
df_read2 = spark.sql("select distinct store_location from test1.clean_transactions1")

In [47]:
df_read2.show()

+--------------+
|store_location|
+--------------+
|    Washington|
|       Houston|
|       NewYork|
|         Miami|
|        Denver|
+--------------+



In [48]:
df_read3 = spark.sql("select * from test1.clean_transactions_default limit 2")

In [49]:
df_read3.show()

+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|store_id|store_location|product_category|product_id|mrp|   cp|discount|   sp|      date|
+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|  YR7220|       NewYork|     Electronics|  12254943| 31|20.77|    1.86|29.14|2019-11-26|
|  YR7220|       NewYork|       Furniture|  72619323| 15| 9.75|     1.5| 13.5|2019-11-26|
+--------+--------------+----------------+----------+---+-----+--------+-----+----------+



## Write Clean Data into PostgreSql

on linux prompt:
[sudo] password for train:
[train@trainvm ~]$ cat /opt/db_conn
[DB]
user_name=train
password=Ankara06
db_ip=127.0.0.1
[train@trainvm ~]$

In [50]:
import configparser

In [53]:
config = configparser.RawConfigParser()

config.read('/opt/db_conn')
user_name = config.get('DB', 'user_name')
password = config.get('DB', 'password')
db_ip = config.get('DB', 'db_ip')

In [54]:
user_name

'train'

In [55]:
df2.show()

+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|store_id|store_location|product_category|product_id|mrp|   cp|discount|   sp|      date|
+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|  YR7220|       NewYork|     Electronics|  12254943| 31|20.77|    1.86|29.14|2019-11-26|
|  YR7220|       NewYork|       Furniture|  72619323| 15| 9.75|     1.5| 13.5|2019-11-26|
|  YR7220|       NewYork|     Electronics|  34161682| 88|62.48|     4.4| 83.6|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  79411621| 91|58.24|    3.64|87.36|2019-11-26|
|  YR7220|       NewYork|         Fashion|  39520263| 85| 51.0|    2.55|82.45|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  93809204| 37|24.05|    0.74|36.26|2019-11-26|
|  YR7220|       NewYork|       Cosmetics|  86610412| 80| 48.8|     6.4| 73.6|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  52503356| 71| 42.6|    5.68|65.32|2019-11-26|
|  YR7220|

In [56]:
df2.printSchema()

root
 |-- store_id: string (nullable = true)
 |-- store_location: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- mrp: integer (nullable = true)
 |-- cp: double (nullable = true)
 |-- discount: float (nullable = true)
 |-- sp: float (nullable = true)
 |-- date: date (nullable = true)



In [57]:
(df2
.write.format("jdbc")
.mode("overwrite")
.option("driver", "org.postgresql.Driver")
.option("url", f"jdbc:postgresql://{db_ip}:5432/traindb")
.option("dbtable", "clean_transactions")
.option("user", user_name)
.option("password", password)
.save())

In [58]:
df_analysis_query = (spark.read.format("jdbc")
.option("driver", "org.postgresql.Driver")
.option("url", f"jdbc:postgresql://{db_ip}:5432/traindb")
.option("query", "select store_location, avg(cp) as avg_cp, avg(discount) AS avg_discount from clean_transactions group by store_location")
.option("user", user_name)
.option("password", password)
.load())

In [59]:
df_analysis_query.show()

+--------------+------------------+------------------+
|store_location|            avg_cp|      avg_discount|
+--------------+------------------+------------------+
|       Houston|37.754843601895644| 3.377819901337556|
|    Washington|37.717426778242505| 3.414211990766362|
|       NewYork| 37.01221354166632|3.2593142362182133|
|        Denver|37.361677360219716|3.2846746087675243|
|         Miami|  36.9078081058724| 3.349412734617962|
+--------------+------------------+------------------+



## Write Data into HDFS

In [60]:
(df2
.write.format("parquet")
.mode("overwrite")
.save("/user/train/spark_odev_transaction"))

In [61]:
!hdfs dfs -ls -t /user/train/spark_odev_transaction

Found 2 items
-rw-r--r--   1 train supergroup          0 2023-02-23 00:31 /user/train/spark_odev_transaction/_SUCCESS
-rw-r--r--   1 train supergroup     241687 2023-02-23 00:31 /user/train/spark_odev_transaction/part-00000-47e03f24-47cb-4ecb-9aa9-c26e11467ec6-c000.snappy.parquet


In [62]:
df_read = (spark.read.format("parquet").load("hdfs://localhost:9000/user/train/spark_odev_transaction"))

In [63]:
df_read.show()

+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|store_id|store_location|product_category|product_id|mrp|   cp|discount|   sp|      date|
+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|  YR7220|       NewYork|     Electronics|  12254943| 31|20.77|    1.86|29.14|2019-11-26|
|  YR7220|       NewYork|       Furniture|  72619323| 15| 9.75|     1.5| 13.5|2019-11-26|
|  YR7220|       NewYork|     Electronics|  34161682| 88|62.48|     4.4| 83.6|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  79411621| 91|58.24|    3.64|87.36|2019-11-26|
|  YR7220|       NewYork|         Fashion|  39520263| 85| 51.0|    2.55|82.45|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  93809204| 37|24.05|    0.74|36.26|2019-11-26|
|  YR7220|       NewYork|       Cosmetics|  86610412| 80| 48.8|     6.4| 73.6|2019-11-26|
|  YR7220|       NewYork|         Kitchen|  52503356| 71| 42.6|    5.68|65.32|2019-11-26|
|  YR7220|

In [64]:
spark.stop()